In [ ]:
# Install & Imports
import os
import json
import random

import torch
from datasets import Dataset
import pandas as pd

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.9.1+cpu
CUDA available: False


In [ ]:
# Load Base Model + Tokenizer
BASE_MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat"  
DATA_PATH = "../data/annotated/training_pairs.jsonl"
OUTPUT_DIR = "models/platelogic_qwen_lora"

os.makedirs(OUTPUT_DIR, exist_ok=True)

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)


In [13]:
# Load JSONL into a pandas DataFrame
records = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            records.append(json.loads(line))

df = pd.DataFrame(records)
print("Total records:", len(df))
df.head(3)


Total records: 300


,input,output
0,Title: granny is licking her chops\nIngredient...,"Season pork chops with salt, pepper, and musta..."
1,Title: crock pot chicken bbq\nIngredients: ['b...,"Season chicken breast with cumin, vinegar, and..."
2,Title: savory smothered pork chops\nIngredient...,Season pork chops with salt and pepper. Sear i...


In [14]:
def build_prompt(example):
    instr = example["input"]
    resp = example["output"]
    text = (
        "You are a cooking assistant that rewrites recipes to be high-protein and low-carb.\n\n"
        "Instruction:\n"
        f"{instr}\n\n"
        "Response:\n"
        f"{resp}"
    )
    return {"text": text}

hf_ds = Dataset.from_pandas(df)
hf_ds = hf_ds.map(build_prompt)

hf_ds = hf_ds.remove_columns([c for c in hf_ds.column_names if c != "text"])
hf_ds = hf_ds.shuffle(seed=SEED)

# Train/validation split
split = hf_ds.train_test_split(test_size=0.1, seed=SEED)
train_ds = split["train"]
val_ds = split["test"]

print(train_ds[0])
print("Train size:", len(train_ds), "Val size:", len(val_ds))


Map: 100%|██████████| 300/300 [00:00<00:00, 2441.67 examples/s]

{'text': "You are a cooking assistant that rewrites recipes to be high-protein and low-carb.\n\nInstruction:\nTitle: sirloin tips with mushrooms\nIngredients: ['sirloin tip steaks', 'butter', 'vegetable oil', 'fresh mushrooms', 'garlic cloves', 'dijon mustard', 'cornstarch', 'heavy cream', 'beef broth', 'white wine vinegar', 'soy sauce', 'dried parsley']\n\nGoal: Rewrite the cooking steps to produce a high-protein, low-carb version while keeping the core flavor.\nConstraints: avoid added sugars, avoid starchy carbs, prioritize lean protein.\n\nWrite the improved version steps.\n\nResponse:\nSeason the main protein and cook until done. Use low-carb sides and avoid added sugars in this sirloin tips with mushrooms recipe."}
Train size: 270 Val size: 30


In [15]:
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_NAME,
    use_fast=True
)

# Some chat models don’t have pad_token; align it with eos_token if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

MAX_LENGTH = 512  # keep this modest for your laptop

def tokenize_function(example):
    out = tokenizer(
        example["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )
    # For simple supervised fine-tuning, labels = input_ids
    out["labels"] = out["input_ids"].copy()
    return out

tokenized_train = train_ds.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_val = val_ds.map(tokenize_function, batched=True, remove_columns=["text"])

tokenized_train.set_format(type="torch")
tokenized_val.set_format(type="torch")

len(tokenized_train), len(tokenized_val)


Map: 100%|██████████| 30/30 [00:00<00:00, 2727.77 examples/s]


(270, 30)

In [16]:
from peft import LoraConfig, get_peft_model, TaskType
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

# ---- choose which modules to apply LoRA to ----
# We focus on attention + MLP projections (standard practice).
candidate_keywords = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj",
    "W_pack"   # for fused qkv, if present
]

target_modules = [
    name.split(".")[-1]
    for name, module in model.named_modules()
    if isinstance(module, nn.Linear)
    and any(kw in name for kw in candidate_keywords)
]

print("Number of target modules:", len(target_modules))
print("Sample target modules:", sorted(set(target_modules))[:15])

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=list(set(target_modules)),
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
model.to(device)


Using device: cpu
Number of target modules: 168
Sample target modules: ['down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj']
trainable params: 3,784,704 || all params: 467,772,416 || trainable%: 0.8091


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=10

In [17]:
BATCH_SIZE = 1 if device == "cpu" else 4

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4 if device == "cpu" else 1,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    warmup_ratio=0.05,
    weight_decay=0.01,
    fp16=(device == "cuda"),
    bf16=False,
    report_to=[],
    logging_dir=os.path.join(OUTPUT_DIR, "logs")
)

def compute_metrics(eval_pred):
    # You can add perplexity later; for now we don’t overcomplicate it.
    return {}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


c:\Users\fatem\Projects\platelogic\.venv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\fatem\AppData\Local\Temp\ipykernel_22752\3844717070.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/67 [00:00<?, ?it/s]c:\Users\fatem\Projects\platelogic\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
 15%|█▍        | 10/67 [04:49<27:45, 29.23s/it]

{'loss': 4.1659, 'grad_norm': 1.666911005973816, 'learning_rate': 0.00018095238095238095, 'epoch': 0.15}


 30%|██▉       | 20/67 [09:49<23:39, 30.20s/it]

{'loss': 1.1246, 'grad_norm': 0.6866673231124878, 'learning_rate': 0.00014920634920634923, 'epoch': 0.3}


 45%|████▍     | 30/67 [14:58<18:54, 30.67s/it]

...

 90%|████████▉ | 60/67 [2:04:57<10:23, 89.05s/it]    

{'loss': 0.3839, 'grad_norm': 0.42747262120246887, 'learning_rate': 2.2222222222222223e-05, 'epoch': 0.89}


100%|██████████| 67/67 [2:08:18<00:00, 114.90s/it]

{'train_runtime': 7698.3476, 'train_samples_per_second': 0.035, 'train_steps_per_second': 0.009, 'train_loss': 1.0906155251744967, 'epoch': 0.99}


TrainOutput(global_step=67, training_loss=1.0906155251744967, metrics={'train_runtime': 7698.3476, 'train_samples_per_second': 0.035, 'train_steps_per_second': 0.009, 'total_flos': 257024738721792.0, 'train_loss': 1.0906155251744967, 'epoch': 0.9925925925925926})

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

PROJECT_ROOT = os.path.abspath("..")  # from notebooks → project root

BASE_MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat"  # same as training

# POINT HERE ⬇️
ADAPTER_DIR = os.path.join(
    PROJECT_ROOT,
    "models",
    "platelogic_qwen_lora",
    "checkpoint-67"
)

print("Base model:", BASE_MODEL_NAME)
print("LoRA adapter dir:", ADAPTER_DIR)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)

model.to(device)
model.eval()

print("✅ Loaded fine-tuned LoRA model.")


Using device: cpu
Base model: Qwen/Qwen1.5-0.5B-Chat
LoRA adapter dir: c:\Users\fatem\Projects\platelogic\models\platelogic_qwen_lora\checkpoint-67
✅ Loaded fine-tuned LoRA model.


In [14]:
import torch
import pandas as pd

# --- Load the training pairs JSONL (this has `input` + `output`) ---
pairs_path = "../data/annotated/training_pairs.jsonl"
pairs = pd.read_json(pairs_path, lines=True)
print("Training pairs loaded:", len(pairs))
print("Columns:", pairs.columns.tolist())

if "input" not in pairs.columns:
    raise ValueError("Expected an 'input' column in training_pairs.jsonl.")

# --- Generation function ---
def generate_recipe(prompt, max_new_tokens=200):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


Training pairs loaded: 300
Columns: ['input', 'output']


In [20]:
sample_input = pairs.iloc[0]["input"]

prompt = (
    "You are a cooking assistant that rewrites recipes to be high-protein and low-carb.\n\n"
    "Instruction:\n"
    f"{sample_input}\n\n"
    "Response:\n"
)

print("=== FULL PROMPT SENT TO MODEL ===\n")
print(prompt[:600])

print("\n=== MODEL OUTPUT ===\n")
print(generate_recipe(prompt, max_new_tokens=200))


=== FULL PROMPT SENT TO MODEL ===

You are a cooking assistant that rewrites recipes to be high-protein and low-carb.

Instruction:
Title: granny is licking her chops
Ingredients: ['pork chops', 'mango chutney', 'granny smith apple', 'sharp cheddar cheese']

Goal: Rewrite the cooking steps to produce a high-protein, low-carb version while keeping the core flavor.
Constraints: avoid added sugars, avoid starchy carbs, prioritize lean protein.

Write the improved version steps.

Response:


=== MODEL OUTPUT ===

You are a cooking assistant that rewrites recipes to be high-protein and low-carb.

Instruction:
Title: granny is licking her chops
Ingredients: ['pork chops', 'mango chutney', 'granny smith apple', 'sharp cheddar cheese']

Goal: Rewrite the cooking steps to produce a high-protein, low-carb version while keeping the core flavor.
Constraints: avoid added sugars, avoid starchy carbs, prioritize lean protein.

Write the improved version steps.

Response:
Season the main protein with 

In [19]:
model.print_trainable_parameters()

trainable params: 0 || all params: 467,772,416 || trainable%: 0.0000
